In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb

In [3]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [6]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    return train_df, test_df, train_labels_df

In [7]:
train_df, test_df, train_labels_df = read_data()

In [8]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [9]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [10]:
train_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


## Feature Engineering

In [23]:
main_key = 'installation_id'
merge_args = {'left_index':True, 'right_index':True}
agg_stats = ['mean', 'sum', 'min', 'max', 'std', 'skew', 'median', pd.Series.kurt, 'sem']

In [24]:
comp_train_df = pd.DataFrame({main_key: train_df[main_key].unique()})
comp_train_df.set_index(main_key, inplace = True)
comp_train_df

""
installation_id
0001e90f
000447c4
0006a69f
0006c192
0009a5a9
...
fff64664
fff70c1e
fff744ce


In [25]:
x_df = train_df.groupby(main_key).agg({'game_time': agg_stats})
x_df.fillna(7)
x_df

game_time                                         \
                          mean        sum min      max            std   
installation_id                                                         
0001e90f         201787.799558  273826044   0   570396  164377.067566   
000447c4          81011.751381   14663127   0   175347   47014.118535   
0006a69f          82491.007366  313548319   0  1520600  159789.192183   
0006c192          90398.352968  201045937   0   395901   77779.338412   
0009a5a9         398593.543689  164220540   0  1684228  648848.534146   
...                        ...        ...  ..      ...            ...   
fff64664         516205.468549  402124060   0  1548496  582787.501688   
fff70c1e          82465.818182    3628496   0   152279   57657.161595   
fff744ce              0.000000          0   0        0       0.000000   
fff944d5          34397.693642    5950801   0    84885   21520.137145   
fffc0583          35545.505976    8921922   0   124887   29508.291970   

                                                              
                     skew    median       kurt           sem  
installation_id                                               
0001e90f         0.664036  150032.0  -0.836821   4462.223352  
000447c4         0.101741   81525.0  -0.937713   3494.531910  
0006a69f         7.504204   50155.0  62.680316   2591.781982  
0006c192         1.366377   69533.0   1.873374   1649.289345  
0009a5a9         1.312675   60399.5  -0.262822  31966.472889  
...                   ...       ...        ...           ...  
fff64664         0.650020   93563.0  -1.241505  20880.529166  
fff70c1e        -0.222848   91783.5  -1.648052   8692.144159  
fff744ce         0.000000       0.0        NaN      0.000000  
fff944d5         0.339054   33049.0  -0.678811   1636.145730  
fffc0583         1.351349   30110.0   1.417156   1862.546874  

[17000 rows x 9 columns]

In [22]:
comp_train_df.merge(x_df, **merge_args)

/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,"(game_time, mean)","(game_time, sum)","(game_time, min)","(game_time, max)","(game_time, std)","(game_time, skew)","(game_time, median)","(game_time, kurt)"
installation_id,,,,,,,,
0001e90f,201787.799558,273826044,0,570396,164377.067566,0.664036,150032.0,-0.836821
000447c4,81011.751381,14663127,0,175347,47014.118535,0.101741,81525.0,-0.937713
0006a69f,82491.007366,313548319,0,1520600,159789.192183,7.504204,50155.0,62.680316
0006c192,90398.352968,201045937,0,395901,77779.338412,1.366377,69533.0,1.873374
0009a5a9,398593.543689,164220540,0,1684228,648848.534146,1.312675,60399.5,-0.262822
...,...,...,...,...,...,...,...,...
fff64664,516205.468549,402124060,0,1548496,582787.501688,0.650020,93563.0,-1.241505
fff70c1e,82465.818182,3628496,0,152279,57657.161595,-0.222848,91783.5,-1.648052
fff744ce,0.000000,0,0,0,0.000000,0.000000,0.0,NaN


In [26]:
dict(train_labels_df.groupby('title')['accuracy_group'].agg(lambda x:x.value_counts().index[0]))

{'Bird Measurer (Assessment)': 0,
 'Cart Balancer (Assessment)': 3,
 'Cauldron Filler (Assessment)': 3,
 'Chest Sorter (Assessment)': 0,
 'Mushroom Sorter (Assessment)': 3}

In [27]:
train_labels_df[[main_key, 'title', 'accuracy_group']]

,installation_id,title,accuracy_group
0,0006a69f,Mushroom Sorter (Assessment),3
1,0006a69f,Bird Measurer (Assessment),0
2,0006a69f,Mushroom Sorter (Assessment),3
3,0006a69f,Mushroom Sorter (Assessment),2
4,0006a69f,Bird Measurer (Assessment),3
...,...,...,...
17685,ffc90c32,Bird Measurer (Assessment),3
17686,ffd2871d,Cauldron Filler (Assessment),3
17687,ffeb0b1b,Cauldron Filler (Assessment),1
17688,ffeb0b1b,Mushroom Sorter (Assessment),0


In [12]:
def get_event_counts(df, column:str='event_id', simple_event_count=True):
    df_event_count = df.groupby([main_key, column]).agg({column: ['count']}) \
    if simple_event_count else df.groupby([main_key, column]).agg({'event_count': 'sum'})
    df_event_count.columns = ['count']
    df_event_count_pivot = df_event_count.pivot_table(index = main_key, columns = [column], values = 'count')
    return df_event_count_pivot.fillna(0)

In [13]:
get_event_counts(train_df, column='event_id', simple_event_count=False)

event_id,003cd2ee,0086365d,00c73085,01ca3a3c,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
installation_id,,,,,,,,,,,,,,,,,,,,,
0001e90f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24357.0,24453.0,311.0,0.0,0.0,0.0,0.0
000447c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,139.0,0.0,0.0,0.0,0.0
0006a69f,0.0,0.0,0.0,0.0,2132.0,6200.0,0.0,1454.0,0.0,0.0,...,2.0,191.0,48.0,3144.0,2907.0,400.0,0.0,523.0,3517.0,0.0
0006c192,0.0,2.0,366.0,0.0,547.0,0.0,363.0,734.0,0.0,0.0,...,1.0,0.0,36.0,1698.0,1493.0,0.0,2.0,1287.0,3816.0,0.0
0009a5a9,0.0,0.0,957.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff64664,0.0,0.0,0.0,0.0,0.0,0.0,8612.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1018.0,1042.0,0.0,4.0,0.0,0.0,0.0
fff70c1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,183.0,194.0,0.0,0.0,0.0,0.0,0.0
fff744ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
get_event_counts(train_df, simple_event_count=True)

event_id,003cd2ee,0086365d,00c73085,01ca3a3c,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
installation_id,,,,,,,,,,,,,,,,,,,,,
0001e90f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,66.0,66.0,4.0,0.0,0.0,0.0,0.0
000447c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
0006a69f,0.0,0.0,0.0,0.0,24.0,70.0,0.0,29.0,0.0,0.0,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
0006c192,0.0,1.0,3.0,0.0,8.0,0.0,9.0,10.0,0.0,0.0,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
0009a5a9,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff64664,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17.0,17.0,0.0,2.0,0.0,0.0,0.0
fff70c1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0
fff744ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def get_event_id_count(df):
    df = df.groupby([main_key]).agg({'event_id': ['count']})
    df.columns = ['event_id_count']
    return df

In [16]:
installation_ids = pd.DataFrame({main_key: train_df[main_key].unique()})
installation_ids.set_index(main_key, inplace = True)
installation_ids

""
installation_id
0001e90f
000447c4
0006a69f
0006c192
0009a5a9
...
fff64664
fff70c1e
fff744ce


In [17]:
event_count_df = get_event_id_count(train_df)
event_count_df

,event_id_count
installation_id,
0001e90f,1357
000447c4,181
0006a69f,3801
0006c192,2224
0009a5a9,412
...,...
fff64664,779
fff70c1e,44
fff744ce,3


In [18]:
installation_ids.merge(event_count_df, **merge_args)

,event_id_count
installation_id,
0001e90f,1357
000447c4,181
0006a69f,3801
0006c192,2224
0009a5a9,412
...,...
fff64664,779
fff70c1e,44
fff744ce,3


In [19]:
get_event_id_count(test_df)

,event_id_count
installation_id,
00abaee7,868
01242218,2719
017c5718,150
01a44906,234
01bc6cb6,952
...,...
fee254cf,212
ff57e602,303
ffc73fb2,526


In [74]:
def get_object_columns(df, column):
    df = df.groupby([main_key, column])['event_id'].count().reset_index()
    df = df.pivot_table(index = main_key, columns = [column], values = 'event_id')
    df.columns = list(df.columns)
    df.fillna(0, inplace = True)
    return df

def get_numeric_columns(df, column):
    df = df.groupby(main_key).agg({f'{column}': agg_stats})
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = [f'{column}_{stat}' for stat in agg_stats]
    return df

def get_numeric_columns_add(df, agg_column, column):
    df = df.groupby([main_key, agg_column]).agg({f'{column}': agg_stats}).reset_index()
    df = df.pivot_table(index = main_key, columns = [agg_column], values = [col for col in df.columns if col not in [main_key, agg_column]])
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = list(df.columns)
    return df

def feature_engineering(train_df, test_df, train_labels_df):
    
    numerical_columns = ['game_time']
    categorical_columns = ['type', 'world']

    comp_train_df = pd.DataFrame({main_key: train_df[main_key].unique()})
    comp_train_df.set_index(main_key, inplace = True)
    comp_test_df = pd.DataFrame({main_key: test_df[main_key].unique()})
    comp_test_df.set_index(main_key, inplace = True)

    for i in numerical_columns:
        comp_train_df = comp_train_df.merge(get_numeric_columns(train_df, i), **merge_args)
        comp_test_df = comp_test_df.merge(get_numeric_columns(test_df, i), **merge_args)
    
    for i in categorical_columns:
        comp_train_df = comp_train_df.merge(get_object_columns(train_df, i), **merge_args)
        comp_test_df = comp_test_df.merge(get_object_columns(test_df, i), **merge_args)
    
    for i in categorical_columns:
        for j in numerical_columns:
            comp_train_df = comp_train_df.merge(get_numeric_columns_add(train_df, i, j), **merge_args)
            comp_test_df = comp_test_df.merge(get_numeric_columns_add(test_df, i, j), **merge_args)
            
    comp_train_df = comp_train_df.merge(get_event_counts(train_df, column='event_id', simple_event_count=True), **merge_args)
    comp_test_df = comp_test_df.merge(get_event_counts(test_df, column='event_id', simple_event_count=True), **merge_args)
    
#     comp_train_df = comp_train_df.merge(get_event_counts(train_df, column='event_code', simple_event_count=True), **merge_args)
#     comp_test_df = comp_test_df.merge(get_event_counts(test_df, column='event_code', simple_event_count=True), **merge_args)
    
#     comp_train_df = comp_train_df.merge(get_event_id_count(train_df), **merge_args)
#     comp_test_df = comp_test_df.merge(get_event_id_count(test_df), **merge_args)
    
    comp_train_df.reset_index(inplace = True)
    comp_test_df.reset_index(inplace = True)
    
    print(f'Our training set has {comp_train_df.shape[0]} rows and {comp_train_df.shape[1]} columns')
    print(f'Our test set has {comp_test_df.shape[0]} rows and {comp_test_df.shape[1]} columns')

    # get the mode of the title
    labels_map = dict(train_labels_df.groupby('title')['accuracy_group'].agg(lambda x:x.value_counts().index[0]))
    # merge target
    labels = train_labels_df[[main_key, 'title', 'accuracy_group']]
    # replace title with the mode
    labels.loc[:,'title'] = labels['title'].map(labels_map)
    # get title from the test set
    comp_test_df.loc[:,'title'] = test_df.groupby(main_key).last()['title'].map(labels_map).reset_index(drop = True)
    # join train with labels
    comp_train_df = labels.merge(comp_train_df, on = main_key, how = 'left')
    print(f'We have {comp_train_df.shape[0]} training rows')
    
    return comp_train_df, comp_test_df

In [75]:
comp_train_df, comp_test_df = feature_engineering(train_df, test_df, train_labels_df)

Our training set has 17000 rows and 755 columns
Our test set has 1000 rows and 736 columns
We have 17690 training rows


In [76]:
pd.options.display.max_columns = None
comp_test_df

,installation_id,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,Activity,Assessment,Clip,Game,CRYSTALCAVES,MAGMAPEAK,NONE,TREETOPCITY,12 Monkeys,Air Show,All Star Sorting,Balancing Act,Bird Measurer (Assessment),Bottle Filler (Activity),Bubble Bath,Bug Measurer (Activity),Cart Balancer (Assessment),Cauldron Filler (Assessment),Chest Sorter (Assessment),Chicken Balancer (Activity),Chow Time,Costume Box,Crystal Caves - Level 1,Crystal Caves - Level 2,Crystal Caves - Level 3,Crystals Rule,Dino Dive,Dino Drink,Egg Dropper (Activity),Fireworks (Activity),Flower Waterer (Activity),Happy Camel,"Heavy, Heavier, Heaviest",Honey Cake,Leaf Leader,Lifting Heavy Things,Magma Peak - Level 1,Magma Peak - Level 2,Mushroom Sorter (Assessment),Ordering Spheres,Pan Balance,Pirate's Tale,Rulers,Sandcastle Builder (Activity),Scrub-A-Dub,Slop Problem,Treasure Map,Tree Top City - Level 1,Tree Top City - Level 2,Tree Top City - Level 3,Watering Hole (Activity),Welcome to Lost Lagoon!,"(game_time, max, Activity)","(game_time, max, Assessment)","(game_time, max, Clip)","(game_time, max, Game)","(game_time, mean, Activity)","(game_time, mean, Assessment)","(game_time, mean, Clip)","(game_time, mean, Game)","(game_time, min, Activity)","(game_time, min, Assessment)","(game_time, min, Clip)","(game_time, min, Game)","(game_time, skew, Activity)","(game_time, skew, Assessment)","(game_time, skew, Clip)","(game_time, skew, Game)","(game_time, std, Activity)","(game_time, std, Assessment)","(game_time, std, Clip)","(game_time, std, Game)","(game_time, sum, Activity)","(game_time, sum, Assessment)","(game_time, sum, Clip)","(game_time, sum, Game)","(game_time, max, CRYSTALCAVES)","(game_time, max, MAGMAPEAK)","(game_time, max, NONE)","(game_time, max, TREETOPCITY)","(game_time, mean, CRYSTALCAVES)","(game_time, mean, MAGMAPEAK)","(game_time, mean, NONE)","(game_time, mean, TREETOPCITY)","(game_time, min, CRYSTALCAVES)","(game_time, min, MAGMAPEAK)","(game_time, min, NONE)","(game_time, min, TREETOPCITY)","(game_time, skew, CRYSTALCAVES)","(game_time, skew, MAGMAPEAK)","(game_time, skew, NONE)","(game_time, skew, TREETOPCITY)","(game_time, std, CRYSTALCAVES)","(game_time, std, MAGMAPEAK)","(game_time, std, NONE)","(game_time, std, TREETOPCITY)","(game_time, sum, CRYSTALCAVES)","(game_time, sum, MAGMAPEAK)","(game_time, sum, NONE)","(game_time, sum, TREETOPCITY)","(game_time, max, 12 Monkeys)","(game_time, max, Air Show)","(game_time, max, All Star Sorting)","(game_time, max, Balancing Act)","(game_time, max, Bird Measurer (Assessment))","(game_time, max, Bottle Filler (Activity))","(game_time, max, Bubble Bath)","(game_time, max, Bug Measurer (Activity))","(game_time, max, Cart Balancer (Assessment))","(game_time, max, Cauldron Filler (Assessment))","(game_time, max, Chest Sorter (Assessment))","(game_time, max, Chicken Balancer (Activity))","(game_time, max, Chow Time)","(game_time, max, Costume Box)","(game_time, max, Crystal Caves - Level 1)","(game_time, max, Crystal Caves - Level 2)","(game_time, max, Crystal Caves - Level 3)","(game_time, max, Crystals Rule)","(game_time, max, Dino Dive)","(game_time, max, Dino Drink)","(game_time, max, Egg Dropper (Activity))","(game_time, max, Fireworks (Activity))","(game_time, max, Flower Waterer (Activity))","(game_time, max, Happy Camel)","(game_time, max, Heavy, Heavier, Heaviest)","(game_time, max, Honey Cake)","(game_time, max, Leaf Leader)","(game_time, max, Lifting Heavy Things)","(game_time, max, Magma Peak - Level 1)","(game_time, max, Magma Peak - Level 2)","(game_time, max, Mushroom Sorter (Assessment))","(game_time, max, Ordering Spheres)","(game_time, max, Pan Balance)","(game_time, max, Pirate's Tale)","(game_time, max, Rulers)","(game_time, max, Sandcastle Builder (Activity))","(game_time, max, Scrub-A-Dub)","(game_time, max, Slop Problem)","(game_time, max, Treasure Map)","(game_time, max, Tree Top City - Level 1)","(game_time, max, Tree Top City - Level 2)","(ga

## Normalize

In [77]:
import re

In [78]:
comp_train_df.columns = [c if type(c) != tuple else '_'.join(c) for c in comp_train_df.columns]

In [79]:
comp_test_df.columns = [c if type(c) != tuple else '_'.join(c) for c in comp_test_df.columns]

In [80]:
comp_train_df.columns = [re.sub(r'\W', '_', s) for s in comp_train_df.columns]

In [81]:
comp_test_df.columns = [re.sub(r'\W', '_', s) for s in comp_test_df.columns]

In [82]:
list(comp_train_df.columns)

['installation_id',
 'title',
 'accuracy_group',
 'game_time_mean',
 'game_time_sum',
 'game_time_min',
 'game_time_max',
 'game_time_std',
 'game_time_skew',
 'Activity',
 'Assessment',
 'Clip',
 'Game',
 'CRYSTALCAVES',
 'MAGMAPEAK',
 'NONE',
 'TREETOPCITY',
 '12_Monkeys',
 'Air_Show',
 'All_Star_Sorting',
 'Balancing_Act',
 'Bird_Measurer__Assessment_',
 'Bottle_Filler__Activity_',
 'Bubble_Bath',
 'Bug_Measurer__Activity_',
 'Cart_Balancer__Assessment_',
 'Cauldron_Filler__Assessment_',
 'Chest_Sorter__Assessment_',
 'Chicken_Balancer__Activity_',
 'Chow_Time',
 'Costume_Box',
 'Crystal_Caves___Level_1',
 'Crystal_Caves___Level_2',
 'Crystal_Caves___Level_3',
 'Crystals_Rule',
 'Dino_Dive',
 'Dino_Drink',
 'Egg_Dropper__Activity_',
 'Fireworks__Activity_',
 'Flower_Waterer__Activity_',
 'Happy_Camel',
 'Heavy__Heavier__Heaviest',
 'Honey_Cake',
 'Leaf_Leader',
 'Lifting_Heavy_Things',
 'Magma_Peak___Level_1',
 'Magma_Peak___Level_2',
 'Mushroom_Sorter__Assessment_',
 'Ordering_Sphe

In [83]:
# def normalize(df: pd.DataFrame, cont_names):
#     "Compute the means and stds of `self.cont_names` columns to normalize them."
#     means, stds = {},{}
#     for n in cont_names:
#         means[n], stds[n] = df[n].mean(), df[n].std()
#         df[n] = (df[n]-means[n]) / (1e-7 + stds[n])

In [84]:
# normalize(comp_train_df, [c for c in comp_train_df.columns if c not in ['installation_id', 'title', 'accuracy_group']])
# normalize(comp_test_df, [c for c in comp_test_df.columns if c not in ['installation_id', 'title', 'accuracy_group']])

## Training

In [85]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [100]:
features = [c for c in comp_train_df.columns if c not in ['accuracy_group', 'installation_id']]
target = 'accuracy_group'

class TrainParams():
    def __init__(self, num_splits=10, max_depth=10, early_stopping_rounds=100):
        self.num_splits = 10
        self.params = {
            'learning_rate': 0.007,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.75,
            "bagging_fraction": 0.8,
            "bagging_seed": 42,
            "max_depth": max_depth,
            'verbose': 1
            'num_leaves': 2**max_depth + 10
        }
        self.early_stopping_rounds = early_stopping_rounds
        
    def __repr__(self):
        return f'num_splits: {self.num_splits} max_depth: {self.params["max_depth"]} early_stopping_rounds: {self.early_stopping_rounds}'
    
class Recorder():
    def __init__(self):
        self.qwk3 = 0.0
        self.lowest_qwk3 = 0.0
        self.best_params = None
        self.best_index = 0
        
    def set_results(self, qwk3, train_params: TrainParams, index):
        if qwk3 > self.qwk3:
            self.qwk3 = qwk3
            self.best_params = train_params
            self.best_index = index
        if qwk3 < self.lowest_qwk3:
            self.lowest_qwk3 = qwk3
    
    def __repr__(self):
        return f'[best]: {self.qwk3} [worst:] {self.lowest_qwk3} {self.best_params} [best index]: {self.best_index}'
    
def partition_train_set(df, factor=0.9):
    rows = df.shape[0]
    train_records = int(rows * factor)
    return df[:train_records], df[train_records:]

def train_model(comp_train_df, train_params: TrainParams, i):
    
    kf = KFold(n_splits=train_params.num_splits, shuffle=False)
    
    oof_pred = np.zeros((len(comp_train_df), 4))
    models = []
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(comp_train_df)):
        print(f'Fold: {fold+1}')
        x_train, x_val = comp_train_df[features].iloc[tr_ind], comp_train_df[features].iloc[val_ind]
        y_train, y_val = comp_train_df[target][tr_ind], comp_train_df[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)

        model = lgb.train(train_params.params, train_set, num_boost_round = 10000, early_stopping_rounds = train_params.early_stopping_rounds, 
                          valid_sets=[train_set, val_set], verbose_eval = train_params.early_stopping_rounds)
        oof_pred[val_ind] = model.predict(x_val)
        models.append(model)
        
        val_crt_fold = qwk3(y_val, oof_pred[val_ind].argmax(axis = 1))
        print(f'Fold: {fold+1} quadratic weighted kappa score: {np.round(val_crt_fold,4)}')
        
    return models, train_params

SyntaxError: invalid syntax (<ipython-input-100-798e03559256>, line 17)

In [101]:
partitioned_train_df, partitioned_valid_df = partition_train_set(comp_train_df, 0.95)
partitioned_train_df.shape, partitioned_valid_df.shape

((16805, 757), (885, 757))

In [102]:
all_models = []
train_params_list = []
num_splits = 10
for i, max_depth in enumerate(range(11, 12)):
    for j, early_stopping_round in enumerate([100]):
        train_params = TrainParams(num_splits=num_splits, max_depth=max_depth, early_stopping_rounds=early_stopping_round)
        print('###', train_params, '###')
        models, train_params = train_model(partitioned_train_df, train_params, j * i)
        all_models.append(models)
        train_params_list.append(train_params)

### num_splits: 10 max_depth: 11 early_stopping_rounds: 100 ###
Fold: 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.95294	valid_1's multi_logloss: 0.988762
[200]	training's multi_logloss: 0.824751	valid_1's multi_logloss: 0.875826
[300]	training's multi_logloss: 0.757488	valid_1's multi_logloss: 0.821596
[400]	training's multi_logloss: 0.714829	valid_1's multi_logloss: 0.78996
[500]	training's multi_logloss: 0.685517	valid_1's multi_logloss: 0.770725
[600]	training's multi_logloss: 0.664674	valid_1's multi_logloss: 0.75842
[700]	training's multi_logloss: 0.64914	valid_1's multi_logloss: 0.75023
[800]	training's multi_logloss: 0.637187	valid_1's multi_logloss: 0.7449
[900]	training's multi_logloss: 0.627687	valid_1's multi_logloss: 0.741622
[1000]	training's multi_logloss: 0.619632	valid_1's multi_logloss: 0.739544
[1100]	training's multi_logloss: 0.613097	valid_1's multi_logloss: 0.738907
[1200]	training's multi_logloss: 0.607376	vali

Fold: 6 quadratic weighted kappa score: 0.6654
Fold: 7
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.956095	valid_1's multi_logloss: 0.992213
[200]	training's multi_logloss: 0.826891	valid_1's multi_logloss: 0.881948
[300]	training's multi_logloss: 0.756888	valid_1's multi_logloss: 0.82561
[400]	training's multi_logloss: 0.714281	valid_1's multi_logloss: 0.794987
[500]	training's multi_logloss: 0.684615	valid_1's multi_logloss: 0.775763
[600]	training's multi_logloss: 0.663709	valid_1's multi_logloss: 0.764512
[700]	training's multi_logloss: 0.648253	valid_1's multi_logloss: 0.756248
[800]	training's multi_logloss: 0.636029	valid_1's multi_logloss: 0.749174
[900]	training's multi_logloss: 0.626347	valid_1's multi_logloss: 0.745143
[1000]	training's multi_logloss: 0.61851	valid_1's multi_logloss: 0.742079
[1100]	training's multi_logloss: 0.611858	valid_1's multi_logloss: 0.73934
[1200]	training's multi_logloss: 0.606325	valid_1's multi_l

In [89]:
def run_predictions(models, df):
    y_pred = np.zeros((len(df), 4))
    for model in models:
        y_pred += model.predict(df[features])
    return y_pred / num_splits

In [90]:
recorder = Recorder()
for i, models in enumerate(all_models):
    valid_df_preds = run_predictions(models, partitioned_valid_df)
    res = qwk3(partitioned_valid_df[target], valid_df_preds.argmax(axis = 1))
    recorder.set_results(res, train_params_list[i], i)

print(recorder)

[best]: 0.6563571095152401 [worst:] 0.0 num_splits: 10 max_depth: 11 early_stopping_rounds: 100 [best index]: 0


## Inference

In [91]:
def add_missing_columns(comp_train_df: pd.DataFrame, comp_test_df: pd.DataFrame):
    missing: set = set(comp_train_df.columns) - set(comp_test_df.columns)
    for col in missing:
        comp_test_df[col] = 0.
    print(f'Added missing colums: {missing}')

In [92]:
add_missing_columns(comp_train_df, comp_test_df)

Added missing colums: {'e4d32835', '119b5b02', '5dc079d8', '1b54d27f', '17ca3959', 'dcb1663e', '7fd1ac25', 'bfc77bd6', 'ecc6157f', '003cd2ee', 'ab4ec3a4', '01ca3a3c', '4074bac2', '2ec694de', '611485c5', '0ce40006', '29a42aea', '13f56524', 'a8cc6fec', 'accuracy_group'}


In [94]:
y_pred = run_predictions(all_models[recorder.best_index], comp_test_df)

In [95]:
assert comp_test_df.shape[0] == y_pred.shape[0]

In [96]:
np.unique(y_pred.argmax(-1), return_counts=True)

(array([0, 1, 2, 3]), array([653,  36,  26, 285]))

In [97]:
def prepare_submission(comp_test_df, y_pred):
    comp_test_df = comp_test_df.reset_index()
    comp_test_df = comp_test_df[['installation_id']]
    comp_test_df['accuracy_group'] = y_pred.argmax(axis = 1)
    sample_submission_df = pd.read_csv(path/'sample_submission.csv')
    sample_submission_df.drop('accuracy_group', inplace = True, axis = 1)
    sample_submission_df = sample_submission_df.merge(comp_test_df, on = 'installation_id')
    sample_submission_df.to_csv('submission.csv', index = False)

In [98]:
prepare_submission(comp_test_df, y_pred)

In [99]:
!head submission.csv

installation_id,accuracy_group
00abaee7,3
01242218,3
017c5718,0
01a44906,0
01bc6cb6,0
02256298,3
0267757a,0
027e7ce5,3
02a29f99,0


In [ ]:
!cat submission.csv | wc -l